<a href="https://colab.research.google.com/github/HireTheHero/Machine_Notebooks/blob/main/vilioexample_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# villo walkthrough
- Environment: colab pro
- Reproducing official procedure in [SCORE_REPRO.md](https://github.com/Muennighoff/vilio/blob/master/SCORE_REPRO.md)

## init

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### installation

In [ ]:
# vilio basics
!cd ../; git clone -b master https://github.com/Muennighoff/vilio.git
!cd ../vilio/py-bottom-up-attention; pip install -r requirements.txt
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!cd ../vilio/py-bottom-up-attention; python setup.py build develop

Cloning into 'vilio'...
remote: Enumerating objects: 2834, done.
remote: Total 2834 (delta 0), reused 0 (delta 0), pack-reused 2834
Receiving objects: 100% (2834/2834), 10.62 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (1492/1492), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-9i6y7cru
  Running command git clone -q https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-9i6y7cru
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
     |████████████████████████████████| 4.0 MB 53.8 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5-py3-none-any.whl size=65332 sha256=b4878342b094d745e0646c2bcf5291d9e06f9e6b29da8aacaa341dd1601ae127
  Stored in directory: /tmp/pip-ephem-wheel-cache-z4qon_ot/wheels/24/1d/09/8167de727fe5b74f832b6fcb5d9069d8f03ca29f337bfe484d
  Created wheel 

In [ ]:
# for hm_pipeline
# !cd /vilio; pip install -r requirements.txt

In [ ]:
# %%writefile new_req.txt
# sacremoses==0.0.43
# pandas==1.1.3
# regex==2020.4.4
# h5py==2.10.0
# filelock==3.0.10
# scipy==1.4.1
# sentencepiece~=0.1.91
# matplotlib==3.2.1
# torch==1.6.0
# tensorflow==2.3.1
# tqdm==4.45.0
# numpy==1.18.1
# six==1.14.0
# packaging==20.1
# wandb==0.10.8
# psutil==5.7.0
# requests==2.23.0
# pytorch_lightning==1.0.4
# ImageHash==4.1.0
# tokenizers~=0.9.2
# transformers==3.5.1 # Required due to some imports in the files under src/vilio/transformers
# torchvision==0.7.0
# jieba==0.42.1
# botocore==1.19.8
# spacy==2.3.2
# boto3==1.16.8
# comet_ml==3.2.5
# dataclasses==0.6
# fairseq==0.9.0
# ftfy==5.8
# fugashi==1.0.5
# ipadic==1.0.0
# lmdb==1.0.0
# Pillow==8.0.1
# py3nvml==0.2.6
# pydantic==1.7.2
# pythainlp==2.2.4
# PyYAML==5.3.1
# scikit_learn==0.23.2
# tensorboardX==2.1
# timeout_decorator==0.4.1
# torchcontrib==0.0.2

In [ ]:
# !pip install -r new_req.txt

In [ ]:
# for hm / lmdb feats
!pip install -q kaggle

### kaggle setup
- [reference](https://www.kaggle.com/general/74235)

In [ ]:
# # %%capture
# from google.colab import files
# from IPython.display import clear_output 
# files.upload()# upload kaggle.json
# clear_output()# in order not to leave json info

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/vilio/kaggle.json ~/.kaggle/
# !cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
iamsouravbanerjee/house-rent-prediction-dataset                House Rent Prediction Dataset                        82KB  2022-08-20 13:49:03           6698        200  1.0              
deepcontractor/mcdonalds-india-menu-nutrition-facts            McDonald's India : Menu Nutrition Dataset             5KB  2022-07-28 17:08:52           5180        181  0.9411765        
arianazmoudeh/airbnbopendata                                   Airbnb Open Data                                     10MB  2022-08-01 15:58:10           3393        103  0.9705882        
iamsouravbanerjee/world-population-dataset                     Wo

In [ ]:
# !pip install --upgrade --force-reinstall --no-deps kaggle

### model setup

In [ ]:
# download the models
## O
!wget https://biglmdiag.blob.core.windows.net/oscar/pretrained_models/large-vg-labels.zip
# ## U
# !wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-large.pt
# ## V
# !wget https://dl.fbaipublicfiles.com/mmf/data/models/visual_bert/visual_bert.pretrained.coco.tar.gz
# ## E
# !wget https://ernie-github.cdn.bcebos.com/model-ernie-vil-large-en.1.tar.gz

--2022-09-04 08:58:09--  https://biglmdiag.blob.core.windows.net/oscar/pretrained_models/large-vg-labels.zip
Resolving biglmdiag.blob.core.windows.net (biglmdiag.blob.core.windows.net)... 52.240.48.36
Connecting to biglmdiag.blob.core.windows.net (biglmdiag.blob.core.windows.net)|52.240.48.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5024784124 (4.7G) [application/octet-stream]
Saving to: ‘large-vg-labels.zip’

large-vg-labels.zip 100%[===================>]   4.68G  14.0MB/s    in 7m 48s  

2022-09-04 09:05:57 (10.2 MB/s) - ‘large-vg-labels.zip’ saved [5024784124/5024784124]



In [ ]:
# !mkdir /vilio
# !mkdir /vilio/data

In [ ]:
# copy to specified directory
## O
!unzip large-vg-labels.zip; cp large-vg-labels/ep_20_590000/pytorch_model.bin /vilio/data
# ## U
# !cp uniter-large.pt /vilio/data
# ## V
# !tar -zxvf visual_bert.pretrained.coco.tar.gz; cp model.pth /vilio/data
# # ## E
# !mkdir /vilio/ernie-vil/data/ernielarge
# !tar -zxvf model-ernie-vil-large-en.1.tar.gz; cp -r model-ernie-vil-large-en/params /vilio/ernie-vil/data/ernielarge

# delete original for memory saving
!rm large-vg-labels.zip; rm -r large-vg-labels; 
# !rm uniter-large.pt
# !rm visual_bert.pretrained.coco.tar.gz; rm config.yaml; rm model.pth
# !rm model-ernie-vil-large-en.1.tar.gz; rm -r model-ernie-vil-large-en

Archive:  large-vg-labels.zip
   creating: large-vg-labels/
   creating: large-vg-labels/ep_20_590000/
 extracting: large-vg-labels/ep_20_590000/added_tokens.json  
  inflating: large-vg-labels/ep_20_590000/config.json  
  inflating: large-vg-labels/ep_20_590000/pytorch_model.bin  
  inflating: large-vg-labels/ep_20_590000/special_tokens_map.json  
  inflating: large-vg-labels/ep_20_590000/vocab.txt  
   creating: large-vg-labels/ep_34_999600/
 extracting: large-vg-labels/ep_34_999600/added_tokens.json  
  inflating: large-vg-labels/ep_34_999600/config.json  
  inflating: large-vg-labels/ep_34_999600/pytorch_model.bin  
  inflating: large-vg-labels/ep_34_999600/special_tokens_map.json  
  inflating: large-vg-labels/ep_34_999600/vocab.txt  
   creating: large-vg-labels/ep_55_1617000/
 extracting: large-vg-labels/ep_55_1617000/added_tokens.json  
  inflating: large-vg-labels/ep_55_1617000/config.json  
  inflating: large-vg-labels/ep_55_1617000/pytorch_model.bin  
  inflating: large-vg-l

### data
- [kaggle reference](https://qiita.com/k_ikasumipowder/items/1c20d8b68dbc94ab2633)

In [ ]:
# # hateful memes
# !unzip /content/drive/MyDrive/vilio/XjiOc5ycDBRRNwbhRlgH.zip -d /vilio/data/
# !unzip /content/drive/MyDrive/vilio/XjiOc5ycDBRRNwbhRlgH.zip -d /vilio/ernie-vil/data/hm/

In [ ]:
# # kaggle sample for testing
# !cp -r /content/drive/MyDrive/vilio/sample/* ../vilio/py*/data/


In [ ]:
# # kaggle hm
# !kaggle datasets download parthplc/facebook-hateful-meme-dataset
# !unzip facebook-hateful-meme-dataset.zip
# !cp -r data/* /vilio/data/
# !cp -r data/* /vilio/ernie-vil/data/hm/
# !cp -r data/img /vilio/py-bottom-up-attention/data

# # delete original for memory saving
# !rm facebook-hateful-meme-dataset.zip

In [ ]:
# # seen/unseen data is required for modeling
# !kaggle datasets download hirethehero/hateful-memes-seenunseen
# !rm train.jsonl
# !cp *.jsonl /vilio/data/

# # delete original for memory saving
# !rm hateful-memes-seenunseen.zip
# !rm *.jsonl

In [ ]:
# # hateful memes from MyDrive
# ## mount
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# hateful memes from MyDrive
!mkdir tmp_data
!unzip /content/drive/MyDrive/vilio/hateful_memes.zip -d tmp_data
!cp -r tmp_data/hateful_memes/* /vilio/data/
# !cp -r tmp_data/hateful_memes/* /vilio/ernie-vil/data/hm/
!cp -r tmp_data/hateful_memes/* /vilio/py-bottom-up-attention/data
!rm -r tmp_data

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: tmp_data/hateful_memes/img/58479.png  
  inflating: tmp_data/hateful_memes/img/58490.png  
  inflating: tmp_data/hateful_memes/img/58491.png  
  inflating: tmp_data/hateful_memes/img/58497.png  
  inflating: tmp_data/hateful_memes/img/58601.png  
  inflating: tmp_data/hateful_memes/img/58602.png  
  inflating: tmp_data/hateful_memes/img/58604.png  
  inflating: tmp_data/hateful_memes/img/58607.png  
  inflating: tmp_data/hateful_memes/img/58609.png  
  inflating: tmp_data/hateful_memes/img/58612.png  
  inflating: tmp_data/hateful_memes/img/58613.png  
  inflating: tmp_data/hateful_memes/img/58621.png  
  inflating: tmp_data/hateful_memes/img/58631.png  
  inflating: tmp_data/hateful_memes/img/58637.png  
  inflating: tmp_data/hateful_memes/img/58640.png  
  inflating: tmp_data/hateful_memes/img/58642.png  
  inflating: tmp_data/hateful_memes/img/58649.png  
  inflating: tmp_data/hateful_memes/img/58672.png  
  inflating: tmp_data/hateful_me

In [ ]:
# kaggle lmdb
!kaggle datasets download muennighoff/hmfeatureszipfin
!unzip hmfeatureszipfin.zip -d /vilio/data/
# !unzip hmfeatureszipfin.zip -d /vilio/ernie-vil/data/hm/

# delete original for memory saving
!rm hmfeatureszipfin.zip

100% 9.56G/9.58G [01:27<00:00, 119MB/s]
100% 9.58G/9.58G [01:27<00:00, 118MB/s]
Archive:  hmfeatureszipfin.zip
  inflating: /vilio/data/detectron.lmdb/data.mdb  
  inflating: /vilio/data/detectron.lmdb/lock.mdb  
Archive:  hmfeatureszipfin.zip
  inflating: /vilio/ernie-vil/data/hm/detectron.lmdb/data.mdb  
  inflating: /vilio/ernie-vil/data/hm/detectron.lmdb/lock.mdb  


### variables and imports

## prep features

### extract image features
- [author's reference](https://www.kaggle.com/muennighoff/vilioexample-nb)

In [ ]:
# !cp -r /content/drive/MyDrive/vilio/sample/* ../vilio/py*/data/
# !cd ../vilio/py-bottom-up-attention; 

# hm_feats.sh
## Attr feats:
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 36 --maxboxes 36
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 50 --maxboxes 50
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
--minboxes 72 --maxboxes 72
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vgattr \
# --minboxes 10 --maxboxes 100

## VG feats:
!cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vg \
--minboxes 50 --maxboxes 50
# !cd ../vilio/py-bottom-up-attention; python detectron2_mscoco_proposal_maxnms.py --batchsize 4 --split img --weight vg \
# --minboxes 10 --maxboxes 100

!cd ../vilio/py*/data; ls

Config '/vilio/py-bottom-up-attention/configs/VG-Detection/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Modifications for VG in ResNet Backbone (modeling/backbone/resnet.py):
	Using pad 0 in stem max_pool instead of pad 1.

Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401

faster_rcnn_from_caffe_attr.pkl: 262MB [00:07, 35.3MB/s]               
100

### copy text/image features to designated directory

In [ ]:
# copy features to dsignated directory
## delete image first
!cp -r /vilio/py*/data/img/ /vilio/data/

# delete original for memory saving
## delete image first
!rm -r /vilio/py*/data/img
# ## backup
# !cp -r /vilio/py*/data /content/drive/MyDrive/vilio

# copy features to dsignated directory
## resume
# !cp -r /vilio/data/img/ /vilio/ernie-vil/data/hm/

!cp /vilio/py*/data/hm_vgattr3636.tsv /vilio/data/
# !cp /vilio/py*/data/hm_vgattr3636.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vgattr3636.tsv

!cp /vilio/py*/data/hm_vgattr5050.tsv /vilio/data/
!rm /vilio/py*/data/hm_vgattr5050.tsv

!cp /vilio/py*/data/hm_vgattr7272.tsv /vilio/data/
# !cp /vilio/py*/data/hm_vgattr7272.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vgattr7272.tsv

# !cp /vilio/py*/data/hm_vgattr10100.tsv /vilio/ernie-vil/data/hm/
# !rm /vilio/py*/data/hm_vgattr10100.tsv

!cp /vilio/py*/data/hm_vg5050.tsv /vilio/data/
# !cp /vilio/py*/data/hm_vg5050.tsv /vilio/ernie-vil/data/hm/
!rm /vilio/py*/data/hm_vg5050.tsv

# !cp /vilio/py*/data/hm_vg10100.tsv /vilio/ernie-vil/data/hm/
# !rm /vilio/py*/data/hm_vg10100.tsv

# delete original for memory saving
# ## delete image first
# !rm -r /vilio/py*/data/img
# ## backup
# !cp -r /vilio/py*/data /content/drive/MyDrive/vilio
## resume
!rm -r /vilio/py*/data/*

In [ ]:
# check list of data
!cd ../vilio/data; ls
# !echo =========================
# !cd ../vilio/ernie-vil/data/hm; ls

detectron.lmdb	  hm_vgattr3636.tsv  pytorch_model.bin	train.jsonl
dev_seen.jsonl	  hm_vgattr5050.tsv  README.md
dev_unseen.jsonl  hm_vgattr7272.tsv  test_seen.jsonl
hm_vg5050.tsv	  img		     test_unseen.jsonl


## hm_pipeline

### additional settings

In [ ]:
%%writefile new_req.txt
sacremoses==0.0.43
pandas==1.1.3
regex==2020.4.4
h5py==2.10.0
filelock==3.0.10
scipy==1.4.1
sentencepiece~=0.1.91
matplotlib==3.2.1
torch==1.6.0
tensorflow==2.3.1
tqdm==4.45.0
numpy==1.18.1
six==1.14.0
packaging==20.1
wandb==0.10.8
psutil==5.7.0
requests==2.23.0
pytorch_lightning==1.0.4
ImageHash==4.1.0
tokenizers~=0.9.2
transformers==3.5.1 # Required due to some imports in the files under src/vilio/transformers
torchvision==0.7.0
jieba==0.42.1
botocore==1.19.8
spacy==2.3.2
boto3==1.16.8
comet_ml==3.2.5
dataclasses==0.6
fairseq==0.9.0
ftfy==5.8
fugashi==1.0.5
ipadic==1.0.0
lmdb==1.0.0
Pillow==8.0.1
py3nvml==0.2.6
pydantic==1.7.2
pythainlp==2.2.4
PyYAML==5.3.1
scikit_learn==0.23.2
tensorboardX==2.1
timeout_decorator==0.4.1
torchcontrib==0.0.2

Writing new_req.txt


In [ ]:
!pip install -r new_req.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 883 kB 7.6 MB/s 
     |████████████████████████████████| 9.5 MB 2.7 MB/s 
     |████████████████████████████████| 679 kB 68.6 MB/s 
     |████████████████████████████████| 2.9 MB 41.7 MB/s 
     |████████████████████████████████| 26.1 MB 93.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.1 MB/s 
     |████████████████████████████████| 12.4 MB 44.7 MB/s 
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 320.4 MB 25 kB/s 
     |████████████████████████████████| 60 kB 9.2 MB/s 
     |████████████████████████████████| 20.1 MB 71.5 MB/s 
     |████████████████████████████████| 1.7 MB 55.1 MB/s 
     |████████████████████████████████| 449 kB 71.2 MB/s 
     |████████████████████████████████| 554 kB 66.2 MB/s 
     |████████████████████████████████| 291 kB 75.0 MB/s 
     |██████████████████████████

In [ ]:
import os
os.kill(os.getpid(), 9)

### run

In [ ]:
# test run
## O
# !cd /vilio; chmod 755 bash/training/O/hm_O.sh; bash bash/training/O/hm_O.sh 20
!cd /vilio; chmod 755 bash/training/O/hm_O.sh; bash bash/training/O/hm_O.sh
# ## U
# !cd /vilio; chmod 755 bash/training/U/hm_U.sh; bash/training/U/hm_U.sh 20
# ## V
# !cd /vilio; chmod 755 bash/training/V/hm_V.sh; bash/training/V/hm_V.sh 20

## E


2022-09-04 13:28:49.889968: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
comet_ml is installed but `COMET_API_KEY` is not set.
pretrain
Load 9136 data from split(s) pretrain.
Start to load Faster-RCNN detected objects from data/HM_img.tsv
Loaded 9136 images in file data/HM_img.tsv in 134 seconds.
Use 9136 data in torch dataset


Some weights of BertOPretraining were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['bert.img_embedding.weight', 'bert.img_embedding.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Load pre-trained model from ./data/pytorch_model.bin
MODIFYING: bert.img_embedding.weight

Weights in loaded but not in model:

Weights in model but not in loaded:
bert.embeddings.position_ids

Batch per epoch: 571
Total Iters: 4568
Warm up Iters: 228
100% 571/571 [10:16<00:00,  1.08s/it]
The train

# EOS